In [2]:
!pwd
!echo $PYTHONPATH

/home/swhors/jupyter-workspace/LSTM/foresttrans
$PWD:/home/swhors/jupyter-workspace/finance:/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-paches


In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import requests
import io
from tqdm import tqdm


def get_lotto_history(round, file_path, verbose=0):
    try:
        # 인덱스를 지정해 시트 설정
        df = pd.read_excel(file_path, engine='openpyxl')
        # 필요한 컬럼 선택 및 이름 변경
        lotto_df = df.iloc[:, [0] + list(range(0, 7))]
        lotto_df.columns = ['round', '1', '2', '3', '4', '5', '6', 'bonus']
        if verbose > 0:
            print(f"최신 데이터 로드 완료. [{round}]")
        return lotto_df
    except requests.exceptions.RequestException as e:
        print(f"데이터를 가져오는 데 실패했습니다: {e}")
        return None


def prepare_data_for_regressor(df, verbose=0):
    """
    로또 데이터를 Regressor가 학습할 수 있는 긴 형태로 변환합니다.
    각 행은 '특정 회차의 특정 번호'를 의미합니다.
    """
    win_numbers_melted = df.melt(id_vars=['round'], value_vars=['1','2','3','4','5','6'], value_name='ball_number')
    
    records = []
    # tqdm은 처리 과정을 시각적으로 보여주는 라이브러리입니다.
    for draw in tqdm(df['round'].unique(), desc="데이터 전처리 중"):
        # 현재 회차의 당첨 번호
        current_win_nums = set(win_numbers_melted[win_numbers_melted['round'] == draw]['ball_number'])
        
        # 이전 회차까지의 데이터
        past_data = win_numbers_melted[win_numbers_melted['round'] < draw]
        
        for ball in range(1, 46):
            # 타겟 변수: 현재 회차에 이 공이 뽑혔는가?
            was_drawn = 1 if ball in current_win_nums else 0
            
            # 피처 계산
            # 1. Frequency (빈도)
            frequency = (past_data['ball_number'] == ball).sum()
            
            # 2. Recency (최근성)
            last_drawn_draws = past_data[past_data['ball_number'] == ball]['round']
            if not last_drawn_draws.empty:
                last_drawn = last_drawn_draws.max()
                recency = draw - last_drawn
            else:
                recency = draw # 한 번도 안 나왔으면 현재 회차만큼의 recency
            
            records.append({
                '회차': draw,
                'ball_number': ball,
                'frequency': frequency,
                'recency': recency,
                'was_drawn': was_drawn # Target
            })
            
    return pd.DataFrame(records)


def predict(regression_data, verbose=0):
    # 3. 학습 데이터 준비
    X = regression_data[['frequency', 'recency']]
    y = regression_data['was_drawn']
    
    # 4. 모델 학습
    if verbose > 0:
        print("\nRandomForestRegressor 모델 학습 중...")
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_regressor.fit(X, y)
    
    # 5. 1176회 예측을 위한 데이터 준비
    if verbose > 0:
        print("\n1176회 예측 데이터 생성 중...")
    predict_input = []
    
    # 1175회까지의 전체 데이터를 기반으로 피처 계산
    full_data_melted = history_df.melt(id_vars=['round'], value_vars=['1','2','3','4','5','6'], value_name='ball_number')
    
    next_draw = 1176
    for ball in range(1, 46):
        frequency = (full_data_melted['ball_number'] == ball).sum()
        last_drawn_draws = full_data_melted[full_data_melted['ball_number'] == ball]['round']
        if not last_drawn_draws.empty:
            last_drawn = last_drawn_draws.max()
            recency = next_draw - last_drawn
        else:
            recency = next_draw
        predict_input.append({'frequency': frequency, 'recency': recency})

    predict_df = pd.DataFrame(predict_input)
    
    # 6. 예측 실행
    predicted_scores = rf_regressor.predict(predict_df)
    
    # 7. 결과 확인
    predict_df['predicted_score'] = predicted_scores
    predict_df['ball_number'] = np.arange(1, 46)
    
    # 점수가 높은 순으로 6개 번호 선택
    top_6_numbers = predict_df.sort_values(by='predicted_score', ascending=False).head(6)
    final_prediction = sorted(top_6_numbers['ball_number'].tolist())
    
    if verbose > 0:
        print("\n--- last prediction ---")
        print(f"Last based on RandomForestRegressor Model : {final_prediction}")
        print("\n* 세부 예측 점수 (상위 10개):")
        print(top_6_numbers.head(10)[['ball_number', 'predicted_score']].to_string(index=False))
    return final_prediction


file_names = [
    (1174, 'xlsx/1173.xlsx'),
    (1175, 'xlsx/1174.xlsx'),
    (1176, 'xlsx/1175.xlsx')
    ]
results = []

for file_name in file_names:
    history_df = get_lotto_history(*file_name)
    if history_df is not None:
        # 2. 데이터 전처리 (시간이 다소 걸릴 수 있습니다)
        regression_data = prepare_data_for_regressor(history_df)
        final_prediction = predict(regression_data)
        results.append((file_name[0], final_prediction))


actual_numbers = [
    [1175, 3, 4, 6, 8, 32, 42],
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
]

for result in results:
    print(result)


데이터 전처리 중: 100%|██████████| 575/575 [00:07<00:00, 71.95it/s]


(1174, [13, 15, 18, 21, 23, 27])
(1175, [6, 13, 15, 18, 21, 32])
(1176, [13, 14, 15, 21, 23, 27])
